In [2]:
import os
from google.colab import userdata

In [11]:
# !pip install fastapi
# !pip install uvicorn
# !pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
#mlflow server --host 127.0.0.1 --port 8080

In [12]:
# Reading WANDB key from colab secrets

os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')
ngrok_token = userdata.get('NGROK_AUTH')
# print(os.environ["WANDB_API_KEY"])
# print(userdata.get('NGROK_AUTH'))


In [13]:

!ngrok authtoken ngrok_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Creating App python file which has two Main components.
1. It will fetch the latest Model from Weights& Bias reprository.
2. Create an endspoint with predict function on the model

In [38]:
%%writefile app.py

import os
import wandb
import sklearn
from joblib import load
from fastapi import FastAPI, HTTPException
from contextlib import asynccontextmanager
from pydantic import BaseModel
import wandb
import joblib
import pandas as pd
import numpy as np

app = FastAPI()

# Define input data schema
class PredictionRequest(BaseModel):
    age: int
    sex: str
    bmi: int
    children: int
    smoker: str
    region: str

# Load the model from WandB
def load_model_from_wandb(project_name: str, artifact_name: str):
    try:
        # Initialize WandB
        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        model_path = artifact.file()  # Assumes a single model file in the artifact
        model = joblib.load(model_path)
        # datadir = artifact.download()
        # pths = datadir + '\\insurance.pkl'
        # model = joblib.load(model)
        run.finish()  # End the WandB run
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")



project_name = "mlops_insurance_price"  # Replace with your project name
artifact_name = "Linear_Model_Insruance_Price2:latest"  # Replace with your artifact name
# artifact_name = "Linear_Model_Insruance_Price2:v0"  # Replace with your artifact name

ml_model = load_model_from_wandb(project_name, artifact_name)

# Prediction endpoint
@app.post("/predict")
def predict(input_data: PredictionRequest):
    try:
        # Convert input data to a dictionary for prediction
        input_dict = input_data.dict()

        df = pd.DataFrame(input_dict, index = [0])

        # Call the model's prediction method
        prediction = ml_model.predict(df)

        # Return the prediction result
        return {f"Estimated Insurance Premium: INR {np.round(prediction[0], 2)} "}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {e}")


Overwriting app.py


Running uvocorn web server in the backend on loacal host and 8000 port

In [39]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 &

nohup: appending output to 'nohup.out'


Checking the uvicorn command logs

In [34]:
!tail nohup.out

wandb: 🚀 View run jumping-dragon-25 at: https://wandb.ai/nitin_swarnkar_ampba2025s-isb/mlops_insurance_price/runs/4aofl5ba
wandb: Find logs at: wandb/run-20241215_065146-4aofl5ba/logs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nitin_swarnkar_ampba2025s (nitin_swarnkar_ampba2025s-isb). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /content/wandb/run-20241215_065222-wts9fa3y
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run apricot-haze-26
wandb: ⭐️ View project at https://wandb.ai/nitin_swarnkar_ampba2025s-isb/mlops_insurance_price
wandb: 🚀 View run at https://wandb.ai/nitin_swarnkar_ampba2025s-isb/mlops_insurance_price/runs/wts9fa3y


Checking if uvicorn server is running

In [47]:
!ps -ax | grep uvicorn

   8628 ?        Sl     0:09 /usr/bin/python3 /usr/local/bin/uvicorn app:app --host 0.0.0.0 --port 8
  20113 ?        S      0:00 /bin/bash -c ps -ax | grep uvicorn
  20115 ?        S      0:00 grep uvicorn


Creating tunnel using ngrok to expose local host url to be visible outside the Colab

In [46]:
from pyngrok import ngrok

# Expose the FastAPI app
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://c5bf-34-125-215-126.ngrok-free.app" -> "http://localhost:8000"
